# Food Delivery Data Analysis Hackathon

## Objective
To combine transactional (CSV), user master (JSON), and restaurant master (SQL script)
data into a single unified dataset and analyze business insights.

The final dataset created is:
**final_food_delivery_dataset.csv**

This dataset acts as the single source of truth for all hackathon questions.

In [1]:
import pandas as pd
import sqlite3
import os

## Step 1: Load Orders Data (CSV)

This file contains transactional order-level data.

In [2]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


## Step 2: Load Users Data (JSON)

This file contains user master data including city and membership type.

In [3]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


## Step 3: Load Restaurants Data (SQL Script)

The restaurants.sql file is a SQL script (not a database).
We create a SQLite database and execute the script.

In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("restaurants.db")
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


## Step 4: Merge All Datasets

Left joins are used to retain all orders.

In [5]:
merged = orders.merge(users, on="user_id", how="left")
merged = merged.merge(restaurants, on="restaurant_id", how="left")

merged.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


## Step 5: Create Final Dataset

This dataset is the single source of truth for all analysis.

In [6]:
merged.to_csv("final_food_delivery_dataset.csv", index=False)

print("final_food_delivery_dataset.csv created successfully.")

final_food_delivery_dataset.csv created successfully.


### Multiple Choice Questions
This section contains MCQs based on the given datasets. Analyze the data carefully and select the correct answers.

### Q1. Which city has the highest total revenue (total_amount) from Gold members?

In [7]:
merged[merged["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

### Q2. Which cuisine has the highest average order value across all orders?

In [8]:
merged.groupby("cuisine")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

### Q3. How many distinct users placed orders worth more than ₹1000 in total?

In [9]:
user_total = merged.groupby("user_id")["total_amount"].sum()
(user_total > 1000).sum()

np.int64(2544)

### Q4. Which restaurant rating range generated the highest total revenue?

In [10]:
merged["rating_range"] = pd.cut(
    merged["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0]
)

merged.groupby("rating_range")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

rating_range
(4.5, 5.0]    2197030.75
(4.0, 4.5]    1960326.26
(3.0, 3.5]    1881754.57
(3.5, 4.0]    1717494.41
Name: total_amount, dtype: float64

### Q5. Among Gold members, which city has the highest average order value?

In [11]:
merged[merged["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

### Q6. Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [12]:
merged.groupby("cuisine").agg(
    distinct_restaurants=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values("distinct_restaurants")

,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


### Q7. What percentage of total orders were placed by Gold members?

In [13]:
total_orders = len(merged)
gold_orders = len(merged[merged["membership"] == "Gold"])

round((gold_orders / total_orders) * 100)

50

### Q8. Which restaurant has the highest average order value but less than 20 total orders?

In [14]:
restaurant_stats = merged.groupby("restaurant_name_x", as_index=False).agg(
    avg_order_value=("total_amount", "mean"),
    order_count=("order_id", "count")
)

restaurant_stats[restaurant_stats["order_count"] < 20] \
    .sort_values("avg_order_value", ascending=False)

,restaurant_name_x,avg_order_value,order_count
173,Hotel Dhaba Multicuisine,1040.222308,13
354,Sri Mess Punjabi,1029.180833,12
236,Ruchi Biryani Punjabi,1002.140625,16
343,Sri Delights Pure Veg,989.467222,18
88,Classic Kitchen Family Restaurant,973.167895,19
...,...,...,...
64,Annapurna Tiffins Punjabi,621.828947,19
126,Darbar Tiffins Non-Veg,596.815556,18
122,Darbar Restaurant Punjabi,589.972857,14
333,Spice Tiffins Pure Veg,578.578667,15


### Q9. Which combination contributes the highest revenue?

In [15]:
merged.groupby(["membership", "cuisine"])["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

### Q10. During which quarter of the year is the total revenue highest?

In [16]:
merged["order_date"] = pd.to_datetime(merged["order_date"])
merged["quarter"] = merged["order_date"].dt.to_period("Q")

merged.groupby("quarter")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_1232\765302383.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  merged["order_date"] = pd.to_datetime(merged["order_date"])


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

### NUMERICAL QUESTIONS ANALYSIS

### Q1. How many total orders were placed by users with Gold membership?

In [17]:
merged[merged["membership"] == "Gold"].shape[0]

4987

### Q2. What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [18]:
round(
    merged[merged["city"] == "Hyderabad"]["total_amount"].sum()
)

1889367

### Q3. How many distinct users placed at least one order?

In [19]:
merged["user_id"].nunique()

2883

### Q4. What is the average order value (rounded to 2 decimals) for Gold members?

In [20]:
round(
    merged[merged["membership"] == "Gold"]["total_amount"].mean(),
    2
)

np.float64(797.15)

### Q5. How many orders were placed for restaurants with rating ≥ 4.5?

In [21]:
merged[merged["rating"] >= 4.5].shape[0]

3374

### Q6. How many orders were placed in the top revenue city among Gold members only?

In [22]:
# Identify top revenue city for Gold members
top_city = (
    merged[merged["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

# Count orders in that city for Gold members
merged[
    (merged["membership"] == "Gold") &
    (merged["city"] == top_city)
].shape[0]

1337

## Conclusion

All datasets were successfully loaded, merged, and validated.
The final dataset will be used to answer all hackathon MCQs.